In [25]:
import re
from typing import NamedTuple
from itertools import permutations

Valve = NamedTuple("Valve", id=int, flowRate=int, neighborValves=list[str])
valveMap = {}
n = 0
with open("16.input", "r") as f:
    for line in f:
        m = re.match(
            r"Valve \b(\w+)\b has flow rate=(\d+); tunnels? leads? to valves? (.*)$",
            line,
        )
        valveName, flowRate, neighborValves = m.groups()
        valveMap[valveName] = Valve(n, int(flowRate), neighborValves.split(", "))
        n += 1

distances = [[float("inf")] * n for _ in range(n)]
for valve in valveMap:
    id, _, neighborValves = valveMap[valve]
    distances[id][id] = 0
    for v in neighborValves:
        distances[id][valveMap[v].id] = 1

# floyd-warshall find lengths of shortest paths between all pairs of valves
for k in range(n):
    for i in range(n):
        for j in range(n):
            distances[i][j] = min(distances[i][j], distances[i][k] + distances[k][j])


In [26]:
def mostPressureCanBeReleased(valves: list[str], maxTime=30):
    """
    returns most pressure can be released in maxTime by open the valves in order
    """
    currentValve = valveMap["AA"].id
    totalPressureReleased = 0
    t = 0
    for v in valves:
        nextValve = valveMap[v].id
        # follow the shortest path to the next valve
        t += distances[currentValve][nextValve]
        if t >= maxTime:
            return totalPressureReleased
        # open the valve
        t += 1
        totalPressureReleased += valveMap[v].flowRate * (maxTime - t)
        currentValve = nextValve
    return totalPressureReleased


pressureReleaseVavles = sorted(v for v in valveMap if valveMap[v].flowRate > 0)
# mostPressure = 0
# for i, valveSeq in enumerate(permutations(pressureReleaseVavles)):
#     pressure = mostPressureCanBeReleased(valveSeq)
#     if pressure > 0:
#         print(i, pressure, valveSeq)
#     mostPressure = max(mostPressure, pressure)

openValves = set()
dfsCache = {}


def dfs(currentValve, remainingTime):
    cacheKey = (currentValve, remainingTime, frozenset(openValves))
    if cacheKey in dfsCache:
        return dfsCache[cacheKey]

    if remainingTime <= 0:
        return 0
    result = 0
    if currentValve != "AA":
        openValves.add(currentValve)
        remainingTime -= 1
        result += valveMap[currentValve].flowRate * remainingTime
    maxPressureReleased = 0
    for v in pressureReleaseVavles:
        if v not in openValves:
            currentValveId = valveMap[currentValve].id
            id = valveMap[v].id
            maxPressureReleased = max(
                maxPressureReleased,
                dfs(v, remainingTime - distances[currentValveId][id]),
            )
    if currentValve != "AA":
        openValves.remove(currentValve)
    result += maxPressureReleased
    dfsCache[cacheKey] = result
    return result


print(dfs("AA", 30))


1641


In [27]:
openValves = set()


def dfs2(currentValve, remainingTime):
    """ DFS for part 2 """
    if remainingTime <= 0:
        # pressure released by the elephant with remaining valves
        pressureReleasedByElephant = dfs("AA", 26)
        return pressureReleasedByElephant
    result = 0
    if currentValve != "AA":
        openValves.add(currentValve)
        remainingTime -= 1
        result += valveMap[currentValve].flowRate * remainingTime

    maxPressureReleased = dfs("AA", 26)  # case when the elephant open remaining valves
    for v in pressureReleaseVavles:
        if v not in openValves:
            currentValveId = valveMap[currentValve].id
            id = valveMap[v].id
            pressureReleased = dfs2(v, remainingTime - distances[currentValveId][id])
            if currentValve == "AA" and pressureReleased > maxPressureReleased:
                print(v, pressureReleased)
            maxPressureReleased = max(maxPressureReleased, pressureReleased)

    if currentValve != "AA":
        openValves.remove(currentValve)

    result += maxPressureReleased
    return result


print(dfs2("AA", 26))


CU 2141
FF 2179
IZ 2261
2261
